In [1]:
import shioaji as sj
from pymongo import MongoClient
from datetime import datetime, timedelta
from pandas import DataFrame
import numpy as np
from Messenger.LineMessenger import LineMessenger as Line
from time import sleep

import warnings
warnings.filterwarnings("ignore")

from prettytable import PrettyTable
import json
import os 
parent = os.path.dirname(os.path.abspath("__file__"))
StrongPath = os.path.join(parent, 'StrongTickers')
if not os.path.isdir(StrongPath):
    os.makedirs(StrongPath)

setting = {
    'user':'kevin83321',
    'pwd':'j7629864',
    'ip':'192.168.2.173',
    'port':'49153'
}

In [2]:
def get_minimum_tick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5
    
def get_commission(price:float, multiplier:int=1000, qty=1, Real:bool=True, direction:str='', dayTrade:bool=False):
    """
    計算個別部位的單邊交易成本

    Params:
        symbol : 商品代碼
        exchange : 交易所
        cost : 交易價格
        multiplier : 價格還原現金之乘數
            例如:
                股票 : 1張 = 1,000股，10元的股票還原現金價值，即為10 *1,000 = 10,000元
                期貨 : 台指期1點200元，假設現在10,000點，則一口台股的價值為 200 * 10,000 = 2,000,000
        qty : 買賣口數或張數
        Real : 是否為實單, default = False
        direction : 交易方向 進場(買賣)或出場
            P.S. 股票交易的交易稅是出場才計算
    """
    tick = get_minimum_tick(price)
    commission = price * (0.1425 / 100) * multiplier * qty
    commission = 20 if commission < 20 else commission
    fee = price * (0.3 / 100) * multiplier * qty
    if dayTrade:
        fee /= 2
    slide = tick * multiplier
    tradeCost = commission * 0.6
    if direction == 'EXIT' or direction == 0:
        tradeCost += fee
    if not Real:
        tradeCost += slide * qty
    return tradeCost

In [3]:
class DataObject:
    
    opening = False
    closing = False
    
    name:str=""
    pre_time:str="09:00:00"
    pre_volume:int = 0
    pre_close:int = 0
    total_v:int = 0
    q20_data:dict = {}
    q80_data:dict = {} 
    open_threshold:float = 0
    _api = None
    symbol:str = ""
    refPrice:float = 0
    entry_percent:float = .06
    exit_percent:float = .09
    sl_ratio:float = .03
    v_threshold:float = .01
    entry_threshold:float = 0
    exit_threshold:float = 0
    max_ret:float = 0
    tmp_ret:float = 0
    pos = 0
    traded = 0
    entry = 0
    entry_time = 0
    order = None
    onOrderProcess = False

    preHigh = None
    preLow = None
    sl_threshold = 0
    
    pnl:float = 0
    
    def __init__(self, contract, open_threshold:float, api=None, PreData:dict=None,
                 v_threshold:float=.01, entry_percent:float=.06, exit_percent:float=.09, 
                 takeprofit:float=.6, start_moving_take = .015, sl_ratio:float=.02, max_size:float = 1):
        self.contract = contract
        self.open_threshold = open_threshold
        self.entry_percent = entry_percent
        self.exit_percent = exit_percent
        self.takeprofit = takeprofit
        self.start_moving_take = start_moving_take
        self.sl_ratio = sl_ratio
        self.v_threshold = v_threshold
        self._initialByContract(contract)
        self.max_size = max_size
        self._api = api
        self.first_5mink = {'open':None, 'high':0,'low':9999, 'close':0}
        if PreData:
            self.preHigh = PreData['PreHigh']
            self.preLow = PreData['PreLow']
        
    def _initialByContract(self, contract):
        self.symbol = contract.code
        self.refPrice = contract.reference
        self.entry_threshold = self.refPrice * (1 + self.entry_percent)
        self.exit_threshold = self.refPrice * (1 + self.exit_percent)
        self.name = contract.name
        
    def updateQ20Dict(self, data:dict={}):

        if data is None:return
        if data['simulate'] :return
        if data['symbol'] != self.symbol: return
        self.q20_data = data
        self.q20_data['timeStr'] = self.q20_data['datetime'].strftime("%H:%M:%S")
        self.updateSignal()
        self.updateStatus()
    
    def updateQ80Dict(self, data:dict):

        if data['symbol'] != self.symbol: return
        self.q80_data = data
    
    def updateOrderDeal(self, stat, data):
        if stat == sj.constant.OrderState.TFTOrder:
            if data['status']['cancel_quantity'] == data['order']['quantity']:
                self.opening = self.closing = onOrderProcess = False
                return
            self.opening = self.pos
            self.closing = not self.pos
        elif type(data) == sj.constant.OrderState.TFTDeal:
            self.pos += (1 if data['action'] == sj.constant.Action.Buy else -1) * data['quantity']
            self.entry = data['price']
            if self.pos:
                self.sl_threshold = self.entry * (1 - self.sl_ratio)
                self.entry_time = signal_time
                self.opening = False
            else:
                exit = data['price']
                signal_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
                print(f"{self.symbol}, close : {close}, exit price : {exit}")
                tmp_pnl = (exit - self.entry) * 1000 * (1 if side == 'S' else -1)
                tmp_pnl -= int(get_commission(self.entry if side == 'S' else exit)) # 進場成本
                tmp_pnl -= int(get_commission(exit if side == 'S' else self.entry, direction='EXIT', dayTrade=True)) # 出場成本
                tmp_ret = tmp_pnl / (self.entry * 1000) # - 1
                self.pnl += tmp_pnl


                cover_text = '\n\n---------------PnL Summary--------------\n'
                cover_text = f'時間 : {datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")}\n'
                cover_text += f'Exit Position of {self.symbol}\n'
                cover_text += f'Entry : {self.entry} at {self.entry_time}\n'
                cover_text += f'Exit : {exit} at {signal_time}\n'
                cover_text += f'Total PnL with Cost : {round(tmp_pnl)}\n'
                cover_text += f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%\n'
                cover_text += f'Total PnL with Cost of max size "{self.max_size}": {self.max_size * round(tmp_pnl)}\n'
                cover_text += f'Trade Value: {self.entry * 1000}\n'
                Line.sendMessage(cover_text)
                print('\n---------------PnL Summary--------------')
                print(f'Exit Position of {self.contract.code}')
                print(f'Entry : {self.entry} at {self.entry_time}')
                print(f'Exit : {exit} at {signal_time}')
                print(f'Total PnL with Cost : {tmp_pnl}')
                print(f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%')
                print(f'Total PnL with Cost of max size {self.max_size}: {self.max_size * tmp_pnl}\n')
                self.traded=True
                self.closing = False
    
    def updateStatus(self):
        """
        Chech position profit loss
        """
        if not self.pos: return
        if self.traded: return
        if self.closing or self.opening: 
#             try:
#                 self._api.update_status()
#                 if self.order is not None:
#                     if self.order.status.status in [sj.constant.Status.PendingSubmit, sj.constant.Status.Submitted]:
#                         self.opening = True
#             except:
#                 pass
            return
        close = self.q20_data['close']
        tmp_pnl = (close - self.entry) * 1000 * self.pos
        tmp_pnl -= int(get_commission(self.entry if self.pos > 0 else close)) # 進場成本
        tmp_pnl -= int(get_commission(close if self.pos > 0 else self.entry, direction='EXIT', dayTrade=True)) # 出場成本
        tmp_ret = tmp_pnl / (self.entry * 1000)
        self.tmp_ret = ((close / self.entry) - 1) * self.pos
        
        if self.tmp_ret > self.max_ret:
            self.sl_threshold = close * (1 - self.sl_ratio)
        self.max_ret = max(self.max_ret, self.tmp_ret)
        # print(self.symbol, self.tmp_ret, self.max_ret, close, self.pos)
        
        do_exit_take = self.tmp_ret >= .01
#         if self.max_ret >= self.start_moving_take:
#             do_exit_take = (self.tmp_ret / self.max_ret) <= (self.takeprofit) if self.max_ret else False
        do_exit_stop = close <= self.sl_threshold
        do_exit = close >= self.exit_threshold if self.pos > 0 else close <= (self.refPrice * (1 - self.exit_percent))
        exit_end = self.q20_data['timeStr'] >= '13:20:00:000000'
        if any([do_exit, do_exit_take, do_exit_stop, exit_end]):
            self.closing=True
            self.DoTrade('S' if self.pos > 0 else 'B')
            self.sendNotifyExit(self.q20_data['datetime'].strftime("%H:%M:%S.%f"), self.symbol, self.name,
                             close, self.q20_data['pct_chg'], do_exit_take or do_exit)
    
    def updateSignal(self):
        # print("updateSignal")
        if not self.q20_data : return
        if self.order: return
        if self.pos: return
        if self.traded: return
        totalV = self.q20_data['totalQty']
        close = self.q20_data['close']
        volume = self.q20_data['qty']
        if self.q20_data['timeStr'] != self.pre_time:
            self.total_v = volume
            self.pre_time = self.q20_data['timeStr']
            self.pre_volume = totalV
        else:
            self.total_v += volume
        if not self.pre_close: 
            self.pre_close = close
            if self.first_5mink['open'] is None:
                self.first_5mink['open'] = close
            self.first_5mink['high'] = max(self.first_5mink['high'], close)
            self.first_5mink['low'] = min(self.first_5mink['low'], close)
            self.first_5mink['close'] = close
            self.pre_volume = totalV
            # print(self.symbol, self.first_5mink)
            return
        
        # print(self.symbol, self.first_5mink, close)
#         v_ratio = self.total_v / self.pre_volume
        self.pre_volume = totalV
        if self.q20_data['timeStr'] < '09:05:00:000000':
            self.first_5mink['high'] = max(self.first_5mink['high'], close)
            self.first_5mink['low'] = min(self.first_5mink['low'], close)
            self.first_5mink['close'] = close
        
        else: 
            if self.q20_data['timeStr'] >= '13:00:00:000000': return
            if all([not self.closing, not self.opening]):
                sig = 0
                if close >= max(self.first_5mink['open'], self.first_5mink['close'], self.preHigh if self.preHigh else 0) * 1.005 and close <= self.entry_threshold:
                    sig = 1
#                 elif close <= min(self.first_5mink['low'], self.preLow if self.preLow else 9999) * .99:
#                     sig = -1
                if sig and not self.opening:
                    self.opening = True
                    self.DoTrade('B' if sig > 0 else 'S')
                    self.sendNotify(self.q20_data['datetime'].strftime("%H:%M:%S.%f"), self.symbol, self.name,
                                 close, self.q20_data['pct_chg'], 'B' if sig > 0 else 'S')
            
                
        self.pre_close = close
        
        
    def DoTrade(self, side):
        close = self.q20_data['close']
        #=========
        # RealTime
        #=========
#         if not self._api:return
#         order_price = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
#         print(f"{self.symbol}, close : {close}, entry price : {order_price}")
#         if abs(order_price / close - 1) >= .005:
#             order_price = close
#         order = self._api.Order(
#             price=order_price,
#             quantity=1,
#             action=sj.constant.Action.Buy if side == 'B' else sj.constant.Action.Sell,
#             price_type=sj.constant.StockPriceType.LMT,
#             order_type=sj.constant.TFTOrderType.ROD,
#             first_sell=sj.constant.StockFirstSell.Yes,# if side == 'S' else sj.constant.StockFirstSell.No,
#         )
#         self.order = self._api.place_order(self.contract, order)
#         if self.order.status.status in [sj.constant.Status.PendingSubmit, sj.constant.Status.Submitted]:
#             self.opening = True
        
        #=========
        # Simulate
        #=========
        if not any([self.opening, self.closing]):return
        if abs(self.pos) > 1: return
        self.pos += 1 if side == 'B' else -1
        signal_time = self.q20_data["datetime"].strftime("%H:%M:%S.%f")
        if self.pos == 0:
            self.traded = True
            exit = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
            print(f"{self.symbol}, close : {close}, exit price : {exit}")
            if abs(exit / close - 1) >= .005:
                exit = close
            tmp_pnl = (exit - self.entry) * 1000 * (1 if side == 'S' else -1)
            tmp_pnl -= int(get_commission(self.entry if side == 'S' else exit)) # 進場成本
            tmp_pnl -= int(get_commission(exit if side == 'S' else self.entry, direction='EXIT', dayTrade=True)) # 出場成本
            tmp_ret = tmp_pnl / (self.entry * 1000) # - 1
            self.pnl += tmp_pnl
            
            
            cover_text = '\n\n---------------PnL Summary--------------\n'
            cover_text = f'時間 : {datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")}\n'
            cover_text += f'Exit Position of {self.symbol}\n'
            cover_text += f'Entry : {self.entry} at {self.entry_time}\n'
            cover_text += f'Exit : {exit} at {signal_time}\n'
            cover_text += f'Total PnL with Cost : {round(tmp_pnl)}\n'
            cover_text += f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%\n'
            cover_text += f'Total PnL with Cost of max size "{self.max_size}": {self.max_size * round(tmp_pnl)}\n'
            cover_text += f'Trade Value: {self.entry * 1000}\n'
            Line.sendMessage(cover_text)
            print('\n---------------PnL Summary--------------')
            print(f'Exit Position of {self.contract.code}')
            print(f'Entry : {self.entry} at {self.entry_time}')
            print(f'Exit : {exit} at {signal_time}')
            print(f'Total PnL with Cost : {tmp_pnl}')
            print(f'Total Ret with Cost : {round(tmp_ret * 100, 2)}%')
            print(f'Total PnL with Cost of max size {self.max_size}: {self.max_size * tmp_pnl}\n')
            
            self.entry = 0
            self.closing = False
            return
        self.entry = self.q80_data['ask1'] if side == 'B' else self.q80_data['bid1']
        print(f"{self.symbol}, close : {close}, entry price : {self.entry}")
        if abs(self.entry / close - 1) >= .005:
            self.entry = close
        self.sl_threshold = self.entry * (1 - self.sl_ratio)
        self.entry_time = signal_time
        self.opening = False
    
    def sendNotify(self, dateStr, idx, name, 
                   close, Ret, Side="B"):#, TVRatio, EVRatio):
        try:
            text = f'時間 : {dateStr}\n'
            text += f'股票代號/名稱 : {idx}/{name}\n'
            if Side == "B":
                text += f'觸發條件 : 突破第一個5分K高點\n'
                text += f'進場方向 : 做多\n'
            else:
                text += f'觸發條件 : 突破第一個5分K低點\n'
                text += f'進場方向 : 做空\n'
            text += f'成交價 : {close}\n'
            
            
            # 漲跌幅量
            text += f'漲跌幅 : {Ret} %\n'
            # text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
            # text += '其他提醒 : \n'
            
            Line.sendMessage(text)
        except Exception as e:
            print(e)

    def sendNotifyExit(self, dateStr, idx, name, 
                   close, Ret, takeprofit=False):
        try:
            text = f'時間 : {dateStr}\n'
            text += f'股票代號/名稱 : {idx}/{name}\n'
            text += f'觸發出場條件 : 若有持倉，已經達到({"停利" if takeprofit else "停損"})條件囉\n'
            text += f'成交價 : {close}\n'
            
            # 漲跌幅量
            text += f'漲跌幅 : {Ret} %\n'
            # text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
            # text += '其他提醒 : \n'
            
            Line.sendMessage(text)
        except Exception as e:
            print('send Exit, Error : ' + e)

In [4]:
def getFollowedAssets(followedAssets=None):
    try:
        if followedAssets is None:
            table = getTable(setting, schema,'StockList')
            updateDate = sorted(table.distinct('UpdateDate'))[-1]
            datas = list(table.find({'Industry':{'$ne':''}}))
            return [x['Ticker'] for x in datas if x['UpdateDate'] == updateDate]
    except Exception as e:
        print(e)
        pass
    else:
        return followedAssets

def getTable(setting, schema, table_name):
    try:
        user = setting['user']
        pwd = setting['pwd']
        ip = setting['ip']
        port = setting['port']
        client = MongoClient(f'mongodb://{user}:{pwd}@{ip}:{port}')
        schema = client['admin'][schema]
        table = schema[table_name]
    except:
        self.sendError('getTable')
    return table

schema = 'TWSE'
# if table_name is None:
table_name = 'historicalPrice'
table = getTable(setting, schema, table_name)

In [5]:
def readStrongTicker(dtStr):
    if os.path.isfile(os.path.join(StrongPath, f'{dtStr}_strongTicker.json')):
        with open(os.path.join(StrongPath, f'{dtStr}_strongTicker.json'), 'r') as f:
            tickers = json.load(f)
            return tickers
    else:
        return {}

def writeTradableTicker(dtStr, tickers):
    with open(os.path.join(StrongPath, f'{dtStr}_tradableTicker.json'), 'w') as f:
        json.dump(tickers,f)
        
def createTradableTable(open_thresholds:dict, max_size_map:dict):
    
    try:
        tickers = []
        max_pos_list = []
        close_list = []
        for ticker in open_thresholds.keys():
            tickers.append(ticker)
            close_list.append(open_thresholds[ticker]['PreClose'])
            max_pos_list.append(max_size_map[ticker])
        table = PrettyTable()
        table.add_column('代號', tickers)
        table.add_column("昨收", close_list)
        table.add_column("最大倉位", max_pos_list)
        table.align = 'r'
    except Exception as e:
        print(e)
    else:
        return table
        
def getStrongTickers(tradeDate = datetime.today(), entry_threshold=.06, v_threshold = 1000):

    open_thresholds = readStrongTicker(tradeDate.strftime("%Y-%m-%d"))
#     print(open_thresholds)
#     return
    if not open_thresholds:
#         _followedAssets = []
#         open_thresholds = {}
        # tickers = self.getFollowedAssets(followedAssets)

        # end_date = datetime(2021,4,27) #.today()
        tickers = getFollowedAssets()
        end_date = tradeDate - timedelta(1)
        start_date = end_date + timedelta(-60)
        table = getTable(setting, schema, table_name)
        datas = list(table.find({'Ticker':{'$in':tickers}, 'Date':{'$gte':start_date.strftime("%Y-%m-%d"), '$lte':end_date.strftime("%Y-%m-%d")}}))
        # datas = readStockDataInter(tickers,start_date, end_date)
        Master_df = DataFrame(datas)
        del Master_df['_id']

        df_map = {}
        for ticker in tickers:
            df_map[ticker] = Master_df[Master_df.Ticker==ticker]

        possible_strong = {}

        for k, df in df_map.items():
            if df.empty: continue
            if df.shape[0] < 20: continue
            # print(df)
            df['Close'] = df.Close.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['High'] = df.High.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Open'] = df.Open.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Low'] = df.Low.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Volume'] = df.Volume.apply(lambda x: 0 if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['MA5'] = df.Close.rolling(5).mean()
            df['MA10'] = df.Close.rolling(10).mean()
            df['MA20'] = df.Close.rolling(20).mean()
            df['DI'] = (df.High+df.Low+2*df.Close) / 4
            df['Amp'] = (df.High-df.Low) / df.Low
            df['AmpStd'] = df.Amp.rolling(5).std()
            df['DiStd'] = df['DI'].rolling(10).std()
            df['VMA5'] = df.Volume.rolling(5).mean()
            # df['MA5Slope'] = df['MA5'].pct_change()
            # df['MA10Slope'] = df['MA10'].pct_change()
            # df['MA20Slope'] = df['MA20'].pct_change()
            MAStd = np.std([df.MA5.iloc[-1]-df.MA10.iloc[-1],df.MA5.iloc[-1]-df.MA20.iloc[-1], df.MA10.iloc[-1]-df.MA20.iloc[-1]])
            open_threshold = max(df['Close'].iloc[-5], df['Close'].iloc[-10], df['Close'].iloc[-20])
            ret_5day = df['Close'].iloc[-1] / df['Close'].iloc[-6] - 1
            if df.VMA5.iloc[-1] / 1000 >= v_threshold and ret_5day >= .15:
#             if (open_threshold / df['Close'].iloc[-1]) < (1 + entry_threshold) and (df.VMA5.iloc[-1]) / 1000 >= v_threshold:
#         #             if df['MA5Slope'].iloc[-1] >= 0.01:# and df['MA10Slope'].iloc[-1] >= 0 and df['MA20Slope'].iloc[-1] >= 0:
#                 temp_ma5 = np.mean(df['Close'].iloc[-4:].tolist() + [open_threshold])
#                 possible_strong[k] = (open_threshold, MAStd, df.DiStd.iloc[-1]/df['Close'].iloc[-1], df.AmpStd.iloc[-1])
#                 _followedAssets.append(k)
                if df['Close'].iloc[-1] <= 100 and df['Close'].iloc[-1] >= 50:
                    open_thresholds[k] = open_threshold
        writeStrongTicker(tradeDate.strftime("%Y-%m-%d"), open_thresholds)
    return open_thresholds

In [6]:
trade_dt = datetime.today() #datetime(2021,12,30)#
trade_dtStr = trade_dt.strftime("%Y-%m-%d")
open_thresholds = readStrongTicker(trade_dtStr)

# open_thresholds

In [7]:
trade_dtStr

'2022-01-04'

In [8]:
api = sj.Shioaji()

In [9]:
api.login("F128497445", "j7629864")

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[FutureAccount(person_id='F128497445', broker_id='F002000', account_id='1473661', signed=True, username='鄭圳宏'),
 Account(account_type=<AccountType.H: 'H'>, person_id='F128497445', broker_id='9A92', account_id='0011645', username='鄭圳宏'),
 StockAccount(person_id='F128497445', broker_id='9A92', account_id='0231901', signed=True, username='鄭圳宏\u3000\u3000')]

In [10]:
from shioaji import TickSTKv1, Exchange, BidAskSTKv1, TickFOPv1, BidAskFOPv1

# @api1.on_tick_stk_v1()
# @api1.on_bidask_stk_v1()
# @api1.on_bidask_fop_v1()
# @api1.on_tick_fop_v1()
def quote_callback(exchange:Exchange, tick:[TickSTKv1, BidAskSTKv1, TickFOPv1, BidAskFOPv1]):
    try:
        NotifyTickers[tick.code] = NotifyTickers.get(tick.code,
                                                DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
                                                api=api, max_size = max_size_map[tick.code], PreData=open_thresholds[tick.code]))
        if type(tick) in [TickSTKv1, TickFOPv1]:
            NotifyTickers[tick.code].updateQ20Dict(dict(
                symbol=tick.code,
                datetime=tick.datetime,
                open=float(tick.open),
                high=float(tick.high),
                low=float(tick.low),
                close=float(tick.close),
                avg_price=float(tick.avg_price),
                qty=int(tick.volume),
                totalQty=int(tick.total_volume),
                pct_chg=float(tick.pct_chg),
                simulate=bool(tick.simtrade),
            ))
#             print(f"Exchange : {exchange}, Tick:{NotifyTickers[ticker].q20_data}\n")
        if type(tick) in [BidAskSTKv1, BidAskFOPv1]:
            NotifyTickers[tick.code].updateQ80Dict(dict(
                symbol = tick.code,
                datetime = tick.datetime,
                bid1 = float(tick.bid_price[0]),
                bid2 = float(tick.bid_price[1]),
                bid3 = float(tick.bid_price[2]),
                bid4 = float(tick.bid_price[3]),
                bid5 = float(tick.bid_price[4]),
                bidQty1 = float(tick.bid_volume[0]),
                bidQty2 = float(tick.bid_volume[1]),
                bidQty3 = float(tick.bid_volume[2]),
                bidQty4 = float(tick.bid_volume[3]),
                bidQty5 = float(tick.bid_volume[4]),
                askQty1 = float(tick.ask_volume[0]),
                askQty2 = float(tick.ask_volume[1]),
                askQty3 = float(tick.ask_volume[2]),
                askQty4 = float(tick.ask_volume[3]),
                askQty5 = float(tick.ask_volume[4]),
                ask1 = float(tick.ask_price[0]),
                ask2 = float(tick.ask_price[1]),
                ask3 = float(tick.ask_price[2]),
                ask4 = float(tick.ask_price[3]),
                ask5 = float(tick.ask_price[4]),
            ))
#             print(f"Exchange : {exchange}, BidAsk : {NotifyTickers[ticker].q80_data}\n")
    except Exception as e:
        print(e)
    
api.quote.set_on_tick_stk_v1_callback(quote_callback)
api.quote.set_on_bidask_stk_v1_callback(quote_callback)
api.quote.set_on_bidask_fop_v1_callback(quote_callback)
api.quote.set_on_tick_fop_v1_callback(quote_callback)

def place_cb(stat, msg):
#     NotifyTickers[msg['code']].updateOrderDeal(stat, msg)
    print(f"Order Status : {stat}, Order Data : {msg}")
    
api.set_order_callback(place_cb)

In [11]:
# remove can't daytrade ticker
tmp_ticker_list = list(open_thresholds.keys())
for ticker in tmp_ticker_list:
    tmp_contract = api.Contracts.Stocks[ticker]
#     print(tmp_contract)
    if tmp_contract.day_trade != sj.constant.DayTrade.No:continue
    open_thresholds.pop(ticker)
    
writeTradableTicker(trade_dtStr, open_thresholds)
    
# seperate capital for each ticker
total_capital = 5e6
seperated_capital = int(total_capital / len(open_thresholds.keys()))
max_size_map = {}
for k, v in open_thresholds.items():
    max_pos = int(seperated_capital / (v['PreClose'] * 1000))
    print(f'{k} 最大倉位 : {max_pos}, 昨收 : {v}')
    max_size_map[k] = max_pos

2323 最大倉位 : 36, 昨收 : {'PreClose': 12.55, 'PreHigh': 12.55, 'PreLow': 11.45}
2359 最大倉位 : 17, 昨收 : {'PreClose': 26.25, 'PreHigh': 26.25, 'PreLow': 24.1}
2495 最大倉位 : 21, 昨收 : {'PreClose': 20.9, 'PreHigh': 20.9, 'PreLow': 18.9}
2614 最大倉位 : 12, 昨收 : {'PreClose': 36.35, 'PreHigh': 36.35, 'PreLow': 35.5}
4720 最大倉位 : 25, 昨收 : {'PreClose': 17.8, 'PreHigh': 17.8, 'PreLow': 15.8}
8104 最大倉位 : 5, 昨收 : {'PreClose': 81.4, 'PreHigh': 83.0, 'PreLow': 76.1}
8261 最大倉位 : 3, 昨收 : {'PreClose': 123.0, 'PreHigh': 125.0, 'PreLow': 116.0}
5299 最大倉位 : 2, 昨收 : {'PreClose': 212.0, 'PreHigh': 214.5, 'PreLow': 194.5}
6156 最大倉位 : 18, 昨收 : {'PreClose': 24.3, 'PreHigh': 24.3, 'PreLow': 22.8}
6532 最大倉位 : 7, 昨收 : {'PreClose': 62.1, 'PreHigh': 62.1, 'PreLow': 59.8}
8450 最大倉位 : 7, 昨收 : {'PreClose': 56.9, 'PreHigh': 56.9, 'PreLow': 53.7}


In [13]:
tradable_text_table = createTradableTable(open_thresholds, max_size_map)
text = f'\n【{trade_dtStr}】當沖策略隔日挑選標的\n\n'
text += tradable_text_table.get_string()
Line.sendMessage(text)

200

In [17]:
NotifyTickers = {}
for i in range(125):
    if i < len(open_thresholds.keys()):
        ticker = sorted(open_thresholds.keys())[i]
        tmp_contract = api.Contracts.Stocks[ticker]
        if tmp_contract.day_trade  != sj.constant.DayTrade.No:
            print("Tradable Ticker : " + ticker)
            # tmp_contract = api1.Contracts.Futures[ticker]
            NotifyTickers[ticker] = NotifyTickers.get(ticker,
                                                      DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
                                                                api=api, max_size = max_size_map[ticker], PreData=open_thresholds[ticker]))
            api.quote.subscribe(
                tmp_contract, 
                quote_type = sj.constant.QuoteType.Tick, # or 'tick'
                version = sj.constant.QuoteVersion.v1 # or 'v1'
            )
            api.quote.subscribe(
                tmp_contract, 
                quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
                version = sj.constant.QuoteVersion.v1 # or 'v1'
            )

Tradable Ticker : 1560
Tradable Ticker : 1599
Tradable Ticker : 1717
Tradable Ticker : 1773
Tradable Ticker : 2323
Tradable Ticker : 2359
Tradable Ticker : 2427
Tradable Ticker : 3094
Tradable Ticker : 3306
Tradable Ticker : 3455
Tradable Ticker : 3550
Tradable Ticker : 3583
Tradable Ticker : 3624
Tradable Ticker : 3672
Tradable Ticker : 4162
Tradable Ticker : 4755
Tradable Ticker : 5452
Tradable Ticker : 6133
Tradable Ticker : 6134
Tradable Ticker : 6148
Tradable Ticker : 6196
Tradable Ticker : 6208
Tradable Ticker : 6667
Tradable Ticker : 6698
Tradable Ticker : 8027
Tradable Ticker : 8028
Tradable Ticker : 8071
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1560 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1560 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/OTC/1599 | Event: Subscribe or Unsubscribe ok

In [18]:
while 1:
    try:
        sleep(1)
    except:
        break

In [19]:
single_pnl = 0
max_pnl = 0
ticker_list = []
pnl_list = []
max_pnl_list = []
max_size_list = []
ret_list = []
est_total_value = 0
est_single_value = 0
for d in NotifyTickers.values():
    ticker_list.append(d.symbol)
    pnl_list.append(int(d.pnl))
    max_size_list.append(d.max_size)
    max_pnl_list.append(int(d.pnl * d.max_size))
    ret_list.append(str(round(d.tmp_ret * 100, 2)) + "%")
    est_single_value += d.open_threshold * 1000
    est_total_value += d.open_threshold * 1000 * d.max_size
    # print(d.symbol, int(d.pnl), str(round(d.tmp_ret * 100, 2)) + "%", d.max_size, int(d.pnl * d.max_size))
    single_pnl += d.pnl
    max_pnl += d.pnl * d.max_size

ticker_list.append("合計")
pnl_list.append(int(single_pnl))
max_size_list.append(est_total_value)
max_pnl_list.append(int(max_pnl))
ret_list.append(str(round((max_pnl / est_total_value) * 100, 2)) + "%")

In [20]:

def createSummaryTable(tickers, pnl_list:list=None, max_pnl_list:list=None, max_size_list:list=None, ret_list:list=None):
    try:
        table = PrettyTable()
        table.add_column('代號', tickers)
        table.add_column("損益", pnl_list)
        table.add_column("報酬率", ret_list)
        table.add_column("最大倉位", max_size_list)
        table.add_column("最大倉損益", max_pnl_list)
        table.align = 'r'
    except Exception as e:
        print(e)
    else:
        return table
    
sum_table = createSummaryTable(ticker_list, pnl_list, max_pnl_list, max_size_list, ret_list)

In [ ]:
sum_table.get_string

In [16]:
text = f'\n【{trade_dtStr}】策略損益\n'
text += f'總損益(單部位) : {int(single_pnl)}\n'
text += f'成交量(單部位) : {est_single_value}\n'
text += f'總報酬(單部位) : {str(round((int(single_pnl) / est_single_value) * 100, 2))}%\n\n'

text += f'總損益(最大倉) : {int(max_pnl)}\n'
text += f'成交量(最大倉) : {est_total_value}\n'
text += f'總報酬(最大倉) : {str(round((int(max_pnl) / est_total_value) * 100, 2))}%\n'
text.split('\n')#.replace('\n', '<br>')

['',
 '【2021-12-29】策略損益',
 '總損益(單部位) : -3418',
 '成交量(單部位) : 578900.0',
 '總報酬(單部位) : -0.59%',
 '',
 '總損益(最大倉) : -26224',
 '成交量(最大倉) : 4651200.0',
 '總報酬(最大倉) : -0.56%',
 '']

In [17]:
trade_result_path = os.path.join(parent, "TradeRecord")
if not os.path.isdir(trade_result_path):
    os.makedirs(trade_result_path)
with open(os.path.join(trade_result_path,f'{trade_dtStr}_ResultTable.html'), 'w') as f:
    f.write(sum_table.get_html_string())

In [18]:
Line.sendMessage(text)

200

In [28]:
readStrongTicker('2021-12-30')

{'1560': {'PreClose': 117.5, 'PreHigh': 117.5, 'PreLow': 107.0},
 '1599': {'PreClose': 50.3, 'PreHigh': 51.7, 'PreLow': 47.55},
 '1616': {'PreClose': 12.75, 'PreHigh': 12.75, 'PreLow': 11.6},
 '1717': {'PreClose': 40.55, 'PreHigh': 41.05, 'PreLow': 37.45},
 '1773': {'PreClose': 178.5, 'PreHigh': 178.5, 'PreLow': 163.0},
 '2323': {'PreClose': 10.45, 'PreHigh': 10.45, 'PreLow': 10.45},
 '2349': {'PreClose': 11.25, 'PreHigh': 11.25, 'PreLow': 10.4},
 '2359': {'PreClose': 21.75, 'PreHigh': 21.75, 'PreLow': 21.75},
 '2427': {'PreClose': 13.3, 'PreHigh': 13.35, 'PreLow': 12.15},
 '3094': {'PreClose': 46.05, 'PreHigh': 46.05, 'PreLow': 42.5},
 '3306': {'PreClose': 36.5, 'PreHigh': 36.5, 'PreLow': 34.0},
 '3455': {'PreClose': 83.6, 'PreHigh': 83.6, 'PreLow': 76.2},
 '3550': {'PreClose': 20.1, 'PreHigh': 20.5, 'PreLow': 19.4},
 '3583': {'PreClose': 98.2, 'PreHigh': 98.2, 'PreLow': 89.8},
 '3624': {'PreClose': 78.1, 'PreHigh': 78.8, 'PreLow': 72.3},
 '3672': {'PreClose': 49.35, 'PreHigh': 49.35,